In [ ]:
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook

# Читаем файл

In [ ]:
path_to_data: Path = Path(r"./test_data/balance.xlsx")

data: pd.DataFrame = pd.read_excel(path_to_data, sheet_name="TDSheet")

# Считываем уровни вложенности из оригинального файла и добавляем в колонку

In [ ]:
wb = load_workbook(path_to_data)
ws = wb["TDSheet"]
outline_levels = [
    ws.row_dimensions[row].outline_level  # type: ignore[index]
    for row in range(ws.min_row + 1, ws.max_row + 1)
]
data["outline_levels"] = outline_levels
# print(
#     f"{ws.cell(row=ws.max_row - 2, column=2).value}",
#     f"{ws.row_dimensions[ws.max_row - 2].outline_level}"
# )

# Добавляем колонку с кодом родителя для каждой группы вложенности

In [ ]:
parents = [-1]
data["parents"] = -1
data.at[data.index[0], "parents"] = parents[-1]
for idx in data.index[:-1]:
    data.at[data.index[idx], "parents"] = parents[-1]
    if data.at[idx, "outline_levels"] < data.loc[idx + 1, "outline_levels"]:
        parents.append(data["Код"].iloc[idx])
    elif data.at[idx, "outline_levels"] > data.at[idx + 1, "outline_levels"]:
        for _ in range(
            data.at[idx, "outline_levels"] - data.at[idx + 1, "outline_levels"]
        ):
            parents.pop()
data.at[data.index[-1], "parents"] = parents[-1]

# Выносим колонку "Код" в индекс

In [ ]:
data = data.set_index("Код")

# Суммируем значение в стобиках по товарам, чтобы потом проверить расчеты

In [ ]:
etalon_sum = data.iloc[:, 1:-2].sum()

# Добавляем строку "Итого"

In [ ]:
sum_df = data.copy()
sum_df.loc[-1, ["Номенклатура", "outline_levels", "parents"]] = ["Итого", 0, -2]
sum_df["parents"] = sum_df["parents"].astype("int")
sum_df["outline_levels"] = sum_df["outline_levels"].astype("int")
sum_df["outline_levels"] = sum_df["outline_levels"].astype("category")

# Суммируем
## Делим по уровням вложенности
## В каждом уровне делим на группы с одинаковыми родителями
## Суммируем группы начиная с самого глубокого уровня вложенности

In [ ]:
outline_groups = sum_df.groupby("outline_levels")
for level in range(sum_df["outline_levels"].cat.categories[-1], -1, -1):  # type: ignore[arg-type]
    level_group = outline_groups.get_group(level)
    parents_group = level_group.groupby("parents")
    for parent, group in parents_group:
        sum_df.loc[parent, sum_df.columns[1:-2]] = group.iloc[:, 1:-2].sum()

# Проверяем правильность суммирования

In [ ]:
sum_for_test = sum_df.loc[-1, sum_df.columns[1:-2]].astype("float")  # type: ignore[index]
pd.testing.assert_series_equal(
    etalon_sum, sum_for_test, check_exact=True, check_names=False
)  # type: ignore[call-overload]

# Сохраним обработанную таблицу целиком в csv

In [ ]:
result_path = path_to_data.with_stem(f"{path_to_data.stem}_result")
sum_df.to_csv(result_path, sep=";", float_format="%.2f", decimal=",")

# Удаляем лишние колонки и копируем данные в буфер обмена

In [ ]:
result = sum_df.drop(columns=["Номенклатура", "outline_levels", "parents"])
result.info()
result.to_clipboard(excel=True, index=False, header=False, float_format="%.2f", decimal=",")